In [3]:

!python3 -c "import sys; print(sys.executable)"
!ls

/home/sabih/Documents/emeritus/may_3/assignments/mlops-student/bin/python3
conda.yml  mlflow_env_vars.sh  mlops-student  mlruns
mlflow.db  MLFlow_Lab.ipynb    MLproject      train.py


# MLFlow lab

### Setting up MLFlow tracking server

We also specify artifact root and backend store URI. This makes it possible to store models.

After running this command tracking server will be accessible at `localhost:5006`

In [7]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 5006 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

### MLProject file

This file is used to configure MLFlow steps.

Using `MLproject` we can define our project's pipeline steps, called *entry points*.

Each entry point in this file corresponds to a shell command.

Entry points can be ran using

```
mlflow run -e <ENTRY_POINT>
```

By default `mlflow run` runs `main` entrypoint.

In [8]:
%cat MLproject

name: basic_mlflow

# this file is used to configure Python package dependencies.
# it uses Anaconda, but it can be also alternatively configured to use pip.
conda_env: conda.yml

# entry points can be ran using `mlflow run <project_name> -e <entry_point_name>
entry_points:
  # download_data:
    # you can run any command using MLFlow
    # command: "bash download_data.sh"
  # MLproject file has to have main entry_point. It can be toggled without using -e option.
  main:
    # parameters is a key-value collection.
    parameters:
      file_name:
        type: str
        default: "day.csv"
      max_k:
        type: int
        default: 10
    command: "python train.py"


First we need to download data. We will use weather data from previous machine learning tutorial.

## Training

Now we can train models. See `train.py`.
It contains code from supervised machine learning tutorial; we added tracking metrics and model.

We will train kNN models for $k \in \{1, 2, ..., 10\}$ using *temperature* and *casual* features.

After running this command you can go to `localhost:5006` and see the trained models.

In [12]:
%%bash
source mlflow_env_vars.sh
mlflow run .

2023/05/08 08:43:34 INFO mlflow.utils.conda: Conda environment mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 already exists.
2023/05/08 08:43:34 INFO mlflow.projects.utils: === Created directory /tmp/tmpkq6uz9zc for downloading remote URIs passed to arguments of type 'path' ===
2023/05/08 08:43:34 INFO mlflow.projects.backend.local: === Running command 'source /home/sabih/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 1>&2 && python train.py' in run with ID '327cc32481f04c1aac0740dae5cb964b' === 


{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]]), 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
 

/home/sabih/anaconda3/envs/mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'sklearn_knn' already exists. Creating a new version of this model...
2023/05/08 08:43:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: sklearn_knn, version 10
Created version '10' of model 'sklearn_knn'.
Registered model 'sklearn_knn' already exists. Creating a new version of this model...
2023/05/08 08:43:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: sklearn_knn, version 11
Created version '11' of model 'sklearn_knn'.
Registered model 'sklearn_knn' already exists. Creating a new version of this model...
2023/05/08 08:43:42 INFO mlflow.tracking._model_registry.client: Waiting up to 30

## Inspecting stored models

The trained models are stored in `mlruns/0`.

These directories contain artifacts and config that is needed to serve them.

In [13]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
cat mlruns/0/$last_model_path/artifacts/knn/MLmodel

artifact_path: knn
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.11.3
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.3.1
model_uuid: 236205d23ce845649b8d7d3079bad324
run_id: 8e509c2c290c426f9b293de1005ee636
utc_time_created: '2023-05-08 03:43:50.294035'


## Serving model

Now that we trained our models we can go to *Models* page on MLFLow UI (http://localhost:5006/#/models).

Click *sklearn_knn* on this page, choose a model and move it to *Production* stage.

The following cell will serve the model at localhost on port 5007.

In [18]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/sklearn_knn/Production -p 5007 --env-manager=conda 



# Prediction

We'll load data that we can feed into prediction server.

Let's predict for first winter day and first non-winter day (first rows of previous two dataframes)

**warning: this might fail at first because the prediction server didn't spin up; in this case wait a minute**

In [22]:
%%bash
data='[[14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]]'
echo $data

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5007/invocations

[[14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   104  100    20  100    84   4960  20833 --:--:-- --:--:-- --:--:-- 34666


{"predictions": [0]}

In [33]:
%%bash
data='[[13.05,1.73,2.04,12.4,92.0,2.72,3.27,0.17,2.91,7.2,1.12,2.91,1150.0]]'
echo $data

curl -d "{\"instances\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5007/invocations

[[14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   107  100    20  100    87   4956  21561 --:--:-- --:--:-- --:--:-- 35666


{"predictions": [0]}

In [40]:
%%bash
data='[[ 12.93,   2.81,   2.7 ,  21.0  ,  96.0  ,   1.54,   0.5 ,   0.53, 0.75,   4.6 ,   0.77,   2.31,600.0]]'
echo $data

curl -d "{\"instances\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5007/invocations

[[ 12.93, 2.81, 2.7 , 21.0 , 96.0 , 1.54, 0.5 , 0.53, 0.75, 4.6 , 0.77, 2.31,600.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   139  100    20  100   119   8054  47925 --:--:-- --:--:-- --:--:-- 69500


{"predictions": [2]}

In [41]:
%%bash
data='[[14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]]'
columns='["alcohol","malic_acid","ash","alcalinity_of_ash","magnesium","total_phenols","flavanoids","nonflavanoid_phenols","proanthocyanins","color_intensity","hue","od280/od315_of_diluted_wines","proline"]'
echo $data
curl -d "{\"dataframe_split\":{\"columns\":$columns, \"data\": $data}}" -H 'Content-Type: application/json' 127.0.0.1:5007/invocations

[[14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
/home/sabih/anaconda3/envs/mlflow-d363261abc9664afc2ad1ec216747581854e4c63/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
100   331  100    20  100   311   3297  51277 --:--:-- --:--:-- --:--:-- 66200


{"predictions": [0]}

In [42]:
%%bash
data='[[14.3,1.711,2.143,15.96,127.0,2.860,3.006,0.22,2.59,5.74,1.04,3.82,1065.01]]'
columns='["alcohol","malic_acid","ash","alcalinity_of_ash","magnesium","total_phenols","flavanoids","nonflavanoid_phenols","proanthocyanins","color_intensity","hue","od280/od315_of_diluted_wines","proline"]'
echo $data
curl -d "{\"dataframe_split\":{\"columns\":$columns, \"data\": $data}}" -H 'Content-Type: application/json' 127.0.0.1:5007/invocations

[[14.3,1.711,2.143,15.96,127.0,2.860,3.006,0.22,2.59,5.74,1.04,3.82,1065.01]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
/home/sabih/anaconda3/envs/mlflow-d363261abc9664afc2ad1ec216747581854e4c63/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
100   336  100    20  100   316   3919  61924 --:--:-- --:--:-- --:--:-- 84000


{"predictions": [0]}

In [45]:
%%bash
data='[[124.23,1.71,2.43,15.6,127.0,0.80,3.06,0.28,2.29,5.64,1.04,3.92,165.0]]'
columns='["alcohol","malic_acid","ash","alcalinity_of_ash","magnesium","total_phenols","flavanoids","nonflavanoid_phenols","proanthocyanins","color_intensity","hue","od280/od315_of_diluted_wines","proline"]'
echo $data
curl -d "{\"dataframe_split\":{\"columns\":$columns, \"data\": $data}}" -H 'Content-Type: application/json' 127.0.0.1:5007/invocations

[[124.23,1.71,2.43,15.6,127.0,0.80,3.06,0.28,2.29,5.64,1.04,3.92,165.0]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
/home/sabih/anaconda3/envs/mlflow-d363261abc9664afc2ad1ec216747581854e4c63/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
100   331  100    20  100   311   6309  98107 --:--:-- --:--:-- --:--:--  107k


{"predictions": [0]}